In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import datetime
import random
import re
import pprint
import json

def getMatchLinks (articleUrl):
    html = urlopen("http://www.espn.co.uk/rugby/fixtures/_/date/" + articleUrl)
    bsObj = bs(html, "lxml")
    tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})
    for t in tst:
        json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                      t.string, flags=re.DOTALL | re.MULTILINE).group(1)

    json_out = json.loads(json_text)
    
    # Get just the match links
    json_match = json_out['schedule']['groups']
    matches = []
    for m in json_match:
        for c in m['complete']:
            matches.append(c['result']['href'])
            
    return matches

links = getMatchLinks('20170304')
links

['/rugby/report?gameId=290088&league=267979',
 '/rugby/report?gameId=290089&league=267979',
 '/rugby/match?gameId=290364&league=270559',
 '/rugby/match?gameId=290363&league=270559',
 '/rugby/match?gameId=290366&league=270559',
 '/rugby/match?gameId=290365&league=270559',
 '/rugby/report?gameId=290524&league=270557',
 '/rugby/report?gameId=290520&league=270557',
 '/rugby/report?gameId=290523&league=270557',
 '/rugby/report?gameId=290786&league=242041',
 '/rugby/report?gameId=290785&league=242041',
 '/rugby/report?gameId=290784&league=242041',
 '/rugby/match?gameId=290783&league=242041',
 '/rugby/report?gameId=290782&league=242041',
 '/rugby/report?gameId=290781&league=242041']

In [2]:
links = getMatchLinks('20160805')
links

['/rugby/match?gameId=290127&league=270555',
 '/rugby/match?gameId=290126&league=270555']

In [3]:
def getStatsLinks (articleUrl):
    html = urlopen("http://www.espn.co.uk" + articleUrl)
    bsObj = bs(html, "lxml")
    tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})
    
    json_out = []
    for t in tst:
        json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                          t.string, flags=re.DOTALL | re.MULTILINE).group(1)
        json_out.append(json.loads(json_text))

    for j in json_out[0]['gamePackage']['links']:
        if j['pageType'] == 'matchstats':
            match_s = j['href']
        elif j['pageType'] == 'playerstats':
            player_s = j['href']
        else match_s, player_s = "", ""
            
    return match_s, player_s

m, p = getStatsLinks(links[0])
print(m)
print(p)

SyntaxError: invalid syntax (<ipython-input-3-9d221f16fb05>, line 17)

In [ ]:
html = urlopen("http://www.espn.co.uk" + links[0])
bsObj = bs(html, "lxml")
tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})

In [ ]:
json_out = []
for t in tst:
    json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                      t.string, flags=re.DOTALL | re.MULTILINE).group(1)
    json_out.append(json.loads(json_text))
    

In [ ]:
for j in json_out[0]['gamePackage']['links']:
    if j['pageType'] == 'matchstats':
        match_s = j['href']
    elif j['pageType'] == 'playerstats':
        player_s = j['href']
        

In [ ]:
def getStats(statsUrl):
    html = urlopen("http://www.espn.co.uk" + statsUrl)
    bsObj = bs(html, "lxml")  
    tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})
    
    stat_out = []
    for t in tst:
        json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                              t.string, flags=re.DOTALL | re.MULTILINE).group(1)
        stat_out.append(json.loads(json_text))
        
    return stat_out[0]

tst = getStats('/rugby/playerstats?gameId=290088&league=267979')


In [ ]:
for t in tst['gamePackage']:
    print(t)

In [ ]:
tst['gamePackage']['gameStrip']

In [ ]:
# home team name from player stats
home = tst['gamePackage']['matchEvents']['away']['name'].title()

# Away team name from player stats:
away = tst['gamePackage']['matchEvents']['home']['name'].title()

# Stadium from player stats:
ground = tst['gamePackage']['matchDetails']['venue']

# Match date from player stats:
dt = tst['gamePackage']['matchDetails']['date']

# Competition from player stats:
comp = tst['gamePackage']['gameStrip']['header']

print ("Home team: ", home)
print ("Away team: ", away)
print("Stadium: ", ground)
print("Date: ", dt)
print("Competition: ", comp)

In [ ]:
match = getStats('/rugby/matchstats?gameId=290088&league=267979')


In [ ]:
for m in match:
    print(m)

In [ ]:
for m in match['gamePackage']:
    print(m)

In [ ]:
match['gamePackage']['matchDetails']

In [ ]:
# Home team from match stats:
h = match['gamePackage']['matchEvents']['home']['name'].title()
# Away team from match stats:
a = match['gamePackage']['matchEvents']['away']['name'].title()
# Date from match stats:
dt = match['gamePackage']['matchDetails']['date']
# Get the date as a date object, from match stats:
from datetime import datetime
as_dt = datetime.strptime(dt, '%I:%M PM, %B %d, %Y')
# Exract the date only poriton
dt_only = as_dt.date()
# Extract the time portion
tm = as_dt.time()
# Year only
yr = as_dt.year
# Month only
mth = as_dt.month
# Day only
dy = as_dt.day
# Competition from match stats
comp = match['gamePackage']['gameStrip']['header']

print("Home:        ", h)
print("Away:        ", a)
print("Date string: ", dt)
print("Date object: ", as_dt)
print("Date only:   ", dt_only)
print("Year:        ", yr)
print("Month:       ", mth)
print("Day:         ", dy)
print("Time:        ", tm)
print("Comp:        ", comp)

In [ ]:
# Parse out the date from the date string
from datetime import datetime
# dt_obj = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')
dt_obj = datetime.strptime('3:00 PM, March 4, 2017', '%I:%M PM, %B %d, %Y')

In [ ]:
# Find the minutes played
tst['gamePackage']['matchCommentary']['events'][4]['text']